## 2) Find the duplicates!


In [1]:
# Import libraries
from collections import defaultdict
import collections
import json

## Step 1: Define hash function

Hash function takes as an input a string.
**Depending if order of characters is important or not, we will be calling hash function without or with the sorted string.**<br/>
Each character is converted into byte representation and then bytes are converted into integer value.<br/>
If same character is appearing several times (n times), integer value of one character is multiplied by number of appereances (n) (i.e. 'A' has value 65 and if it appears 3 times it would be represented with 195)<br/>
Values for each character are concatenated. Result is long string containing numbers (i.e. 'AAAB' is represented with 19566).<br/>
In order to get a hash value, modulo operation with prime number 100000007 is applied on a password number representation.
This way we have a range of possible values close to the number of passwords in input file.


In [6]:
def hash_fun(ins):
    val='' # initialize a list for concatenation of the int values for each character
    i=1
    prev=ins[0] # initialize to the first character of the password
    count=1 # count how many times element appears in the password
    while i <= len(ins):
        ne=ins[i:i+1] # read the next character of the password
        if prev==ne: # if next is equal to previous add 1 to the counter
            count+=1
        else: # when new character is different from the previous convert the previous into it's number representation
            b=int.from_bytes((bytes(prev, 'utf-8')), byteorder='big')*count 
            count=1 # reset counter
            prev=ne # reset previous character
            val+=str(b) # concatenate value of the character to the list
        i+=1
    return(int(val)%100000007) # retrun hash value

**Definition A**: Two passwords are considered to be the same if they have the same characters, order is not important. Example: "AABA" = "AAAB"

## Step 2: Create hash map

We used "Defaultdict" structure in Python to store the hash map. <br/>
For each hash value we are storing the list of sorted passwords which belong to that value using our hash function. <br/>
At this point we chose to store sorted passwords to be able to easily use the "Counter" function from "Collections". <br/> 
Alternatively, we could have stored passwords in original form and do the sorting afterwards. In that scenario we would be able also to produce a full file with all duplicated passwords (duplicated in the sense of the **Definiton A**).

hash_map = defaultdict(list)

In [4]:
with open('D:\\Various documents\\Masters\\La Sapienza\\ADM\\Homework\\Homework 4\\passwords2.txt','r') as f:
    for line in f:
            i=hash_fun(''.join(sorted(line.strip('\n')))) # Calling hash function for sorted passwords
            hash_map[str(i)].append(''.join(sorted(line.strip('\n')))) # Storing sorted password in the hash value position
        

In [10]:
# Our hash function created 632144532 unique values for passwords from input file
len(hash_map)

63214532

Save hash map into the file.

In [ ]:
with open('pwd_hash.txt', 'w') as file:
     file.write(json.dumps(hash_map)) # use`json.loads` to do the reverse

## Step 3: Count duplicates and false positives

Per Definition A, **duplicates** are passowrds which have same characters, no matter the order of the characters. They should have the same hash value.<br/>
**False positives** are passowrds which have the same hash value although they are not having the same characters. False positives appear due to the fact that our hash function is not "1-1" transformation. <br/>

Function **find_true** is used to detect which password from the list of ones with the same hash value will be considered the true one.
We decided to take password with the highest frequency (i.e. the most duplicates) as the true one. All the rest are false positive.

In [10]:
def find_true(cnt): # function takes a Collections.Counter object - {value:frequency}
    true_val=0
    true_el=''
    for key,val in cnt.items(): # iterate through the counter and take one with the highest frequency
            if(val>true_val):
                true_val=val
                true_el=key
    return(true_el)

In [50]:
# Passwords are the same if they have same characters no matter the order
s=0 # number of times that same passwords appear
u=0 # number of unique passwords
fpu=0 # number of unique false positive
fp=0 # number of times that false positive happened
for key in hash_map.keys():
    cnt=collections.Counter(hash_map[key]) # for each item in the default dictionary calculate how many times it appears
    l=len(cnt.keys()) # number of unique passwords
    # Ideally, only 1 unique password should be allocated to one hash value
    # All the rest on the same hash value should be considered as false positive
    fpu += (l-1)
    e=find_true(cnt) # finding password which will be considered as true one for the given hash value
    for key, val in cnt.items(): # iterate through the counts and count duplicates and false positives
        if(val>=2): # count duplicates
            s+=val
            u+=1
        if(key!=e): # count false positives
            fp+=val
print(u, 'passwords are duplicated and they appear in total ', s ,' times.')
print(fpu, 'unique passwords could be considered as false positive and they appear in total ', fp, ' times.')

10000000 passwords are duplicated and they appear in total  20000000  times.
36785468 unique passwords could be considered as false positive and they appera in total  37268765  times.


**Definition B**: Two passwords are considered to be the same if they have the same characters, in the same order. Example: "AABA" = "AABA"

## Step 2: Create hash map


We used "Defaultdict" structure in Python to store the hash map. <br/>
For each hash value we are storing the list of passwords which belong to that value using our hash function. <br/>

In [3]:
hash_mapB = defaultdict(list)

In [7]:
with open('D:\\Various documents\\Masters\\La Sapienza\\ADM\\Homework\\Homework 4\\passwords2.txt','r') as f:
    for line in f:
            i=hash_fun(line.strip('\n')) # Calling hash function for passwords
            hash_mapB[str(i)].append(line.strip('\n')) # Storing password in the hash value position
        

In [8]:
# Our hash function created 65009019 unique values for passwords from input file
len(hash_mapB)

65009019

Save hash map into the file.

In [ ]:
with open('pwd_hash.txt', 'w') as file:
     file.write(json.dumps(hash_mapB)) # use`json.loads` to do the reverse

## Step 3: Count duplicates and false positives

Per Definition B, duplicates are only passowrds which are completly the same strings. They should have the same hash value.
False positives are passowrds which have the same hash value although they are not excatly the same. False positives appear due to the fact that our hash function is not "1-1" transformation. 

In [11]:
# Passwords are the same if they have same characters no matter the order
s=0 # number of times that same passwords appear
u=0 # number of unique passwords
fpu=0 # number of unique false positive
fp=0 # number of times that false positive happened
for key in hash_mapB.keys():
    cnt=collections.Counter(hash_mapB[key]) # for each item in the default dictionary calculate how many times it appears
    l=len(cnt.keys()) # number of unique passwords
    # Ideally, only 1 unique password should be allocated to one hash value
    # All the rest on the same hash value should be considered as false positive
    fpu += (l-1)
    e=find_true(cnt) # finding password which will be considered as true one for the given hash value
    for key, val in cnt.items(): # iterate through the counts and count duplicates and false positives
        if(val>=2): # count duplicates
            s+=val
            u+=1
        if(key!=e): # count false positives
            fp+=val
print(u, 'passwords are duplicated and they appear in total ', s ,' times.')
print(fpu, 'unique passwords could be considered as false positive and they appear in total ', fp, ' times.')

5000000 passwords are duplicated and they appear in total  10000000  times.
39990981 unique passwords could be considered as false positive and they appear in total  40113613  times.
